# Prédiction prix au m² — Appartements 2020 (clean + Pipeline + joblib)

**Objectif :** notebook final propre, **sans LightGBM**, avec une **Pipeline sklearn sérialisable** (joblib) incluant le feature engineering `nb_ventes_commune` (anti-leakage).

- Dataset : `../data/prod/df_model_appart_2020.parquet.gz`
- Cible : `prix_m2`
- Pipeline : `CommuneSalesTransformer` → `SimpleImputer` → modèle


In [7]:
# 📦 Imports & settings
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import joblib
from pathlib import Path

RANDOM_STATE = 42
DATA_PATH = "../data/prod/df_model_appart_2020.parquet.gz"
MODEL_DIR = Path("../data/models")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format


## 1) Chargement + nettoyage minimal

In [8]:
df = pd.read_parquet(DATA_PATH, engine="pyarrow")
print("Shape:", df.shape)
display(df.head(3))
display(df["prix_m2"].describe())

# Nettoyage minimal
df = df.dropna(subset=["prix_m2"]).copy()
df = df[df["prix_m2"] > 0].copy()

# Encodage boolean
if "has_dependance" in df.columns:
    df["has_dependance"] = df["has_dependance"].astype(int)

print("After minimal cleaning:", df.shape)


Shape: (190522, 7)


,surface_reelle_bati,nombre_pieces_principales,latitude,longitude,has_dependance,nom_commune,prix_m2
0,62.00,3.00,46.20,5.22,True,Bourg-en-Bresse,"2,193.55"
1,47.00,2.00,46.31,4.84,True,Saint-Laurent-sur-Saône,"1,531.91"
2,46.00,2.00,46.21,5.22,False,Bourg-en-Bresse,"1,521.74"


count   190,522.00
mean      3,687.87
std       2,594.67
min         465.00
25%       1,950.00
50%       2,962.96
75%       4,461.54
max      14,166.67
Name: prix_m2, dtype: float64

After minimal cleaning: (190522, 7)


## 2) Définition cible / features (mêmes noms)
On garde `nom_commune` **uniquement** pour fabriquer `nb_ventes_commune` dans la pipeline.

In [9]:
FEATURES_BASE = [
    "surface_reelle_bati",
    "nombre_pieces_principales",
    "latitude",
    "longitude",
    "has_dependance",
]
TARGET = "prix_m2"

X = df[FEATURES_BASE + ["nom_commune"]].copy()
y = df[TARGET].copy()

display(X.head(3))


,surface_reelle_bati,nombre_pieces_principales,latitude,longitude,has_dependance,nom_commune
0,62.00,3.00,46.20,5.22,1,Bourg-en-Bresse
1,47.00,2.00,46.31,4.84,1,Saint-Laurent-sur-Saône
2,46.00,2.00,46.21,5.22,0,Bourg-en-Bresse


## 3) Split train/test (avant tout feature engineering)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print("Train:", X_train.shape, "Test:", X_test.shape)


Train: (152417, 6) Test: (38105, 6)


## 4) Baseline + métriques

In [11]:
def regression_report(y_true, y_pred, label="model"):
    rmse = root_mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{label:>18} | RMSE: {rmse:,.0f} | MAE: {mae:,.0f} | R²: {r2:,.3f}")
    return {"rmse": rmse, "mae": mae, "r2": r2}

# Baseline = médiane du train
y_pred_base = np.full(shape=len(y_test), fill_value=float(y_train.median()))
baseline_metrics = regression_report(y_test, y_pred_base, "Baseline(median)")


  Baseline(median) | RMSE: 2,681 | MAE: 1,772 | R²: -0.075


## 5) Pipeline propre (feature engineering inclus)
On encapsule `nb_ventes_commune` dans un transformer sklearn pour pouvoir faire `joblib.dump(pipe)`.

In [12]:
class CommuneSalesTransformer(BaseEstimator, TransformerMixin):
    """Ajoute nb_ventes_commune (calculée sur le train) puis retourne les features numériques finales."""

    def __init__(self, commune_col="nom_commune", base_features=None, fill_strategy="median"):
        self.commune_col = commune_col
        self.base_features = base_features or []
        self.fill_strategy = fill_strategy

    def fit(self, X, y=None):
        X_ = X.copy()
        counts = X_.groupby(self.commune_col).size()
        self.commune_sales_ = counts.to_dict()
        self.median_sales_ = float(counts.median()) if len(counts) else 0.0
        return self

    def transform(self, X):
        X_ = X.copy()
        X_["nb_ventes_commune"] = X_[self.commune_col].map(self.commune_sales_)
        fill_value = self.median_sales_ if self.fill_strategy == "median" else 0.0
        X_["nb_ventes_commune"] = X_["nb_ventes_commune"].fillna(fill_value)

        features_final = self.base_features + ["nb_ventes_commune"]
        return X_[features_final]

FEATURES_FINAL = FEATURES_BASE + ["nb_ventes_commune"]

fe = CommuneSalesTransformer(
    commune_col="nom_commune",
    base_features=FEATURES_BASE,
    fill_strategy="median"
)

pipe_rf = Pipeline(steps=[
    ("fe", fe),
    ("imputer", SimpleImputer(strategy="median")),
    ("model", RandomForestRegressor(
        n_estimators=200,
        max_depth=20,
        min_samples_leaf=20,
        random_state=RANDOM_STATE,
        n_jobs=-1,
    ))
])

pipe_gbr = Pipeline(steps=[
    ("fe", fe),
    ("imputer", SimpleImputer(strategy="median")),
    ("model", GradientBoostingRegressor(random_state=RANDOM_STATE))
])

pipe_rf


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('fe', ...), ('imputer', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,commune_col,'nom_commune'
,base_features,"['surface_reelle_bati', 'nombre_pieces_principales', ...]"
,fill_strategy,'median'
,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'median'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",None
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True


## 6) Entraînement + évaluation (test)

In [13]:
results = []

for name, pipe in [("Pipeline_RF", pipe_rf), ("Pipeline_GBR", pipe_gbr)]:
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    m = regression_report(y_test, y_pred, name)
    results.append({"model": name, **m})

results_df = pd.DataFrame(results).sort_values("rmse")
display(results_df)


       Pipeline_RF | RMSE: 1,056 | MAE: 685 | R²: 0.833
      Pipeline_GBR | RMSE: 1,360 | MAE: 968 | R²: 0.723


,model,rmse,mae,r2
0,Pipeline_RF,"1,056.23",685.09,0.83
1,Pipeline_GBR,"1,360.24",967.94,0.72


## 7) Cross-validation (anti-crash)
⚠️ Pour éviter le kernel qui plante : `cross_val_score(..., n_jobs=1)`.
Le RF garde `n_jobs=-1` pour paralléliser en interne.

In [14]:
cv = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

best_name = results_df.iloc[0]["model"]
best_pipe = pipe_rf if best_name == "Pipeline_RF" else pipe_gbr

cv_rmse = -cross_val_score(
    best_pipe,
    X_train,
    y_train,
    scoring="neg_root_mean_squared_error",
    cv=cv,
    n_jobs=1
)

print("Best:", best_name)
print(f"CV RMSE mean: {cv_rmse.mean():,.0f} | std: {cv_rmse.std():,.0f}")


Best: Pipeline_RF
CV RMSE mean: 1,088 | std: 9


## 8) Entraîner le meilleur sur tout le dataset + joblib.dump

In [ ]:
best_pipe.fit(X, y)

ARTIFACTS_DIR = Path("../data/models")
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

# 1️⃣ Modèle RandomForest entraîné
joblib.dump(
    model,
    ARTIFACTS_DIR / "random_forest_prix_m2.joblib"
)

# 2️⃣ Mapping nb ventes par commune (Series)
joblib.dump(
    commune_sales,
    ARTIFACTS_DIR / "sales_per_commune.joblib"
)

# 3️⃣ Valeur médiane de fallback
joblib.dump(
    median_sales,
    ARTIFACTS_DIR / "median_sales.joblib"
)

print("✅ Artefacts sauvegardés dans :", ARTIFACTS_DIR.resolve())


NameError: name 'model' is not defined

## 9) Chargement + prédiction 


In [16]:
loaded_pipe = joblib.load(MODEL_DIR / "prix_m2_pipeline_2020.joblib")
preds = loaded_pipe.predict(X_test.head(5))

display(pd.DataFrame({
    "y_true": y_test.head(5).values,
    "y_pred": preds
}))


,y_true,y_pred
0,"2,689.82","3,024.89"
1,"1,887.32","1,265.25"
2,"3,883.33","2,542.64"
3,696.20,"1,484.65"
4,"2,807.02","2,871.47"


## 10) Focus Paris (optionnel)

In [17]:
pattern_exclude = "Seyssinet-Pariset|Le Touq|Fontenay-en-Parisis|Cormeilles-en-Parisis"

df_paris = df[
    df["nom_commune"].str.contains("Paris", case=False, na=False)
    & ~df["nom_commune"].str.contains(pattern_exclude, case=False, na=False)
].copy()

print("Nb lignes Paris:", len(df_paris))

X_paris = df_paris[FEATURES_BASE + ["nom_commune"]].copy()
y_paris = df_paris[TARGET].copy()

y_pred_paris = loaded_pipe.predict(X_paris)
regression_report(y_paris, y_pred_paris, "BestPipe on Paris")


Nb lignes Paris: 12233
 BestPipe on Paris | RMSE: 2,067 | MAE: 1,473 | R²: 0.299


{'rmse': 2066.845046973825, 'mae': 1472.951873441402, 'r2': 0.2989420785734491}